In [14]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
X_ant = np.load('./data/G_21_ant.npy')
X_grapes = np.load('./data/G_21_grapes.npy')

Les data sets représentent des images dessinées en noir et blanc. Ils contiennent chacun $1000$ lignes et $784$ colonnes : $(1000, 784)$. Chaque colonne représente la couleur d'un pixel sur une échelle de gris de $0$ à $255$. Les $1000$ lignes correspondent à $1000$ images différentes. Les images ont une taille de $28\times28$, d'où les $784$ colonnes.

# Creates labels for ant and grapes

ant := label $0$ and grapes := label 1

In [3]:
y_ant = np.ones((X_ant.shape[0],))
y_grapes = -np.ones((X_grapes.shape[0],))

# Merge the data into one matrix

In [4]:
X = np.concatenate((X_ant, X_grapes), axis=0)

In [5]:
y = np.concatenate((y_ant, y_grapes), axis=0)

# Separate the data into test, validate and train set

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Use SVM to separate data

## SVC with linear kernel

In [7]:
clf_lin = SVC(kernel='linear', C=1)
clf_lin.fit(X_train, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [8]:
clf_lin.score(X_test, y_test)

0.734

## SVC with gaussian kernel

In [9]:
clf_rbf = SVC(kernel='rbf', C=1)
clf_rbf.fit(X_train, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
clf_rbf.score(X_test, y_test)

0.864

## Use cross validation to choose the best parameters for the linear kernel (C) and for the gaussian kernel ($\gamma$ & C)

# Use perceptron to separate the data

In [12]:
clf_p = Perceptron(tol=1e-3, random_state=1)
clf_p.fit(X_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=1, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [13]:
clf_p.score(X_test, y_test)

0.69